In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
def calcRelVort(pltdatau, pltdatav, griddata):
    uy = pltdatau.diff('eta_rho').values*griddata.pn[:-1,:-1]
    uy = 0.5*(uy[1:,:] + uy[:-1,:])
    vx = pltdatav.diff('xi_rho').values*griddata.pm[:-1,:-1]
    zeta = vx[:-1,:]-uy
    cori = 2*2*np.pi/86400*np.sin(griddata.lat_rho*np.pi/180)
    zetanorm = zeta/cori[1:-1,:-1]
    ny, nx = griddata.pm.shape
    temp = 0*np.zeros((ny, nx))
    temp[1:-1,1:]  = zetanorm
    return temp

In [ ]:
files_loc = 'enter/path/here/'
grid = xr.open_dataset(files_loc+'croco_grd.nc')
his = xr.open_mfdataset(files_loc+'croco_his*.nc', concat_dim='time', combine='nested', data_vars='minimal')
yearday = parent_his.scrum_time/86400

In [ ]:
slevel = 49
clevs = np.linspace(-1,1,50)

time = np.arange(len(yearday.values))
for ti in time:
    print(ti)

    vort = calcRelVort(his.u.isel(time=ti, s_rho=slevel), his.v.isel(time=ti, s_rho=slevel), grid)

    plt.figure(figsize=(15, 10), dpi=300)

    plt.contourf(grid.lon_rho.values, grid.lat_rho.values, vort.T,, clevs, cmap='seismic', extend='both')

    cb = plt.colorbar()
    cb.set_label('Vorticity [$\zeta/f$]', fontsize=20)
    cb.set_ticks([-1, 0, 1])
    cb.ax.tick_params(labelsize=20)

    plt.annotate('doy={:.0f}'.format(yearday[ti].values % 360), xy=(0.02, 0.95), xycoords='axes fraction', bbox=dict(facecolor='white', edgecolor='black'))


    plt.savefig('plots/vort_'+str(ti)+'.png', bbox_inches='tight')
    plt.close('all')